In [ ]:
import cv2
import numpy as np
from collections import deque
import pickle
import dill

# lsFilename = ['video/cam2_cls_0.3.mp4']
# lsFilename = ['video/cam2_cls_inf.mp4']
# lsFilename = ['video/cam2_far_0.3.mp4']
lsFilename = ['video/cam2_far_inf.mp4']


nFrame4calib = 100
bShow = True
# bShow = False
bRandomSample = True
# bRandomSample = False
# delay = 150
delay = 10
# delay = 0

nCorners = (5,7)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((nCorners[0]*nCorners[1],3), np.float32)
objp[:,:2] = np.mgrid[0:nCorners[0],0:nCorners[1]].T.reshape(-1,2) * 0.0288

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
frames = []

valid_frames = []
valid_grays = []
valid_corners = []

In [ ]:
nFrame = 0
for filename in lsFilename:
    cap = cv2.VideoCapture(filename)
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")

    nFrame += int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print 'total :',nFrame

In [ ]:
n = 0
for filename in lsFilename:
    print filename
    cap = cv2.VideoCapture(filename)
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
        continue

    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            find, corners = cv2.findChessboardCorners(gray, nCorners, None)
            n += 1
            print n,
            if find:
                valid_frames.append(frame)
                valid_grays.append(gray)
                valid_corners.append(corners)
                print "find ",len(valid_frames),
            print
        else: 
            break
    cap.release()

nFrame = len(valid_frames)
print "Detected %d frames" % nFrame

filename = 'globalsave1.pkl'
dill.dump_session(filename)


In [1]:
import cv2
import numpy as np
from collections import deque
import pickle
import dill

filename = 'globalsave1.pkl'
dill.load_session(filename)


In [2]:

if nFrame <= nFrame4calib:
    idxs = range(nFrame)
else:
    if bRandomSample:
        idxs = np.sort(np.random.choice(nFrame, nFrame4calib, replace=False)).tolist()
    else:
        idxs = np.linspace(0,nFrame-1,nFrame4calib).astype(int).tolist()
# idxs = range(0,len(valid_frames),3)

print len(idxs)
for idx in idxs:
    print "frame ",idx
    frame = valid_frames[idx]
    gray = valid_grays[idx]
    corners = valid_corners[idx]

    objpoints.append(objp)
    corners = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
    imgpoints.append(corners)
    cv2.drawChessboardCorners(frame, nCorners, corners, find)
    frames.append(frame) 
    if bShow:
        cv2.imshow('Frame',frame)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()

cali_ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

print mtx
print dist

print len(idxs)
print len(rvecs)
print len(tvecs)

# np.savetxt('out.txt', np.append(mtx, dist))

filename = 'globalsave2.pkl'
dill.dump_session(filename)


100
frame  2
frame  6
frame  15
frame  32
frame  35
frame  39
frame  43
frame  64
frame  78
frame  102
frame  161
frame  162
frame  170
frame  187
frame  189
frame  193
frame  220
frame  225
frame  243
frame  248
frame  262
frame  268
frame  288
frame  302
frame  305
frame  327
frame  336
frame  362
frame  378
frame  392
frame  400
frame  412
frame  418
frame  427
frame  434
frame  437
frame  441
frame  466
frame  472
frame  488
frame  495
frame  497
frame  498
frame  513
frame  514
frame  542
frame  551
frame  557
frame  558
frame  572
frame  585
frame  587
frame  614
frame  625
frame  639
frame  643
frame  649
frame  656
frame  662
frame  665
frame  681
frame  713
frame  731
frame  735
frame  736
frame  745
frame  748
frame  749
frame  750
frame  761
frame  772
frame  802
frame  813
frame  828
frame  846
frame  849
frame  851
frame  875
frame  907
frame  909
frame  933
frame  934
frame  954
frame  962
frame  1012
frame  1018
frame  1031
frame  1034
frame  1063
frame  1067
frame  1074

In [ ]:
import cv2
import numpy as np
from collections import deque
import pickle
import dill

filename = 'globalsave2.pkl'
dill.load_session(filename)


In [ ]:
idxs_test = range(0,len(valid_frames),10)

######################### test ##########
objpoints_test = [] # 3d point in real world space
imgpoints_test = [] # 2d points in image plane.
frames_test = []
print len(idxs_test)
for idx in idxs_test:
    print "frame ",idx
    frame = valid_frames[idx]
    gray = valid_grays[idx]
    corners = valid_corners[idx]

    objpoints_test.append(objp)
    corners = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
    imgpoints_test.append(corners)
    cv2.drawChessboardCorners(frame, nCorners, corners, find)
    frames_test.append(frame) 
    # if bShow:
    #     cv2.imshow('Frame',frame)
    #     if cv2.waitKey(delay) & 0xFF == ord('q'):
    #         break

cv2.destroyAllWindows()


cali_ret, _, _, rvecs_test, tvecs_test = cv2.calibrateCamera(objpoints_test, imgpoints_test, 
                                                             gray.shape[::-1], mtx, dist)




In [ ]:
bShow = True
delay = 150


tot_error = 0
tot_error_undist = 0
mean_error = 0


def MyProjection(objp, rvec, tvec, mtx, dst):
    rot, _ = cv2.Rodrigues(rvec)
    imgp = np.dot(rot,objp.T) + tvec
    imgp = imgp / imgp[2,:]

    x = imgp[0,:]
    y = imgp[1,:]
    
    r = np.sqrt(x**2 + y**2)

    k1, k2, p1, p2, k3 = dst
    c = (1 + k1*r**2 + k2*r**4 + k3*r**6)/(1)
    x_new = c*x + 2*p1*x*y + p2*(r**2 + 2*x**2)
    y_new = c*y + p1*(r**2 + 2*y**2) + 2*p2*x*y

    imgp[0,:]= x_new
    imgp[1,:]= y_new
    

    imgp = np.dot(mtx,imgp)[:2,:].T.astype(objp.dtype)
    return imgp


# for frame, rvec, tvec, corners, objp in zip(frames_test, rvecs_test, tvecs_test, imgpoints_test, objpoints_test):
for i in range(len(frames_test)):
    frame, rvec, tvec, corners, objp = frames_test[i], rvecs_test[i], tvecs_test[i], imgpoints_test[i], objpoints_test[i]
    imgp1 = MyProjection(objp, rvec, tvec, mtx, dist[0])
    # imgpoints1, _ = cv2.projectPoints(objp, rvec, tvec, mtx, dist)
    # print cv2.norm(imgpoints1.reshape(imgp1.shape), imgp1, cv2.NORM_L2)
    
    for p in imgp1:
        cv2.circle(frame, tuple(p.astype(int)), 1, (0,0,255), 3)

    tot_error += cv2.norm(corners,imgp1.reshape(corners.shape), cv2.NORM_L2)/len(imgp1)

    if bShow:
        cv2.imshow('Frame',frame)
        # cv2.imshow('Dst',dst)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
           break

print "total error: ", tot_error/len(objpoints)






